<a href="https://colab.research.google.com/github/Aleo0/RuyaChatbot/blob/main/RuyaTabir_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import sys
from google.colab import files, drive, userdata
import pandas as pd
import re
import time
from IPython.display import display, Markdown
import textwrap

# Kütüphaneleri kontrol et ve yükle
def install_if_not_exists(package):
    if not any(package in p for p in sys.modules):
        %pip install {package} --quiet
        print(f"{package} yüklendi.")
    else:
        print(f"{package} zaten yüklü.")

install_if_not_exists("chromadb")
install_if_not_exists("sentence_transformers")
install_if_not_exists("google-generativeai")
install_if_not_exists("pandas")

from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings
from chromadb import PersistentClient
from chromadb.utils import embedding_functions
import google.generativeai as genai

# Markdown formatlama fonksiyonu
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Google Drive'a bağlan
drive.mount('/content/drive')
chromaDB_path = '/content/drive/MyDrive/Colab Notebooks/RuyaTabiriDB'

# ChromaDB istemci oluşturma
def create_chroma_client(collection_name, embedding_function, chromaDB_path):
    chroma_client = PersistentClient(
        path=chromaDB_path,
        settings=Settings(),
        tenant=DEFAULT_TENANT,
        database=DEFAULT_DATABASE
    )
    chroma_collection = chroma_client.get_or_create_collection(
        name=collection_name,
        embedding_function=embedding_function
    )
    return chroma_client, chroma_collection

# Excel dosyasını yükle ve ChromaDB'ye ekle
def load_excel_to_chromadb(collection_name, sentence_transformer_model, chromaDB_path, file_path):
    df = pd.read_excel(file_path)
    if 'Rüya' not in df.columns or 'Yorum' not in df.columns:
        raise ValueError("Excel dosyasında 'Rüya' ve 'Yorum' sütunları olmalı.")
    ruyalar = df['Rüya'].tolist()
    yorumlar = df['Yorum'].tolist()
    embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
        model_name=sentence_transformer_model
    )
    chroma_client, chroma_collection = create_chroma_client(
        collection_name, embedding_function, chromaDB_path
    )
    ids = [str(i) for i in range(len(ruyalar))]
    metadatas = [{'yorum': yorum} for yorum in yorumlar]
    print(f"Koleksiyona {len(ruyalar)} rüya-yorum çifti ekleniyor...")
    chroma_collection.add(ids=ids, documents=ruyalar, metadatas=metadatas)
    print(f"Koleksiyonda toplam {chroma_collection.count()} öğe var.")
    return chroma_client, chroma_collection

# Mevcut ChromaDB'yi yükle
def load_existing_chromadb(collection_name, sentence_transformer_model, chromaDB_path):
    embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
        model_name=sentence_transformer_model
    )
    chroma_client, chroma_collection = create_chroma_client(
        collection_name, embedding_function, chromaDB_path
    )
    if chroma_collection.count() == 0:
        raise ValueError("Koleksiyon boş! Lütfen önce Excel dosyasını yükleyin.")
    print(f"Mevcut koleksiyon yüklendi: {chroma_collection.count()} öğe.")
    return chroma_client, chroma_collection

# Rüyaları sorgula
def retrieve_docs(chroma_collection, query, n_results=5):
    results = chroma_collection.query(
        query_texts=[query],
        n_results=n_results,
        include=['documents', 'metadatas', 'distances']
    )
    return results

# Filtreleme fonksiyonu
def filter_relevant_result(docs, metadatas, distances):
    return docs[0][0], metadatas[0][0]['yorum'], distances[0][0]

# Safety settings
from google.generativeai.types.safety_types import HarmCategory, HarmBlockThreshold

safety_settings = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
}

# Gemini LLM bağlantısı
def build_chatbot(system_instruction, api_key):
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('gemini-2.0-flash',
                                 system_instruction=system_instruction,
                                 safety_settings=safety_settings)
    chat = model.start_chat(history=[])
    return chat

def generate_llm_answer(prompt, context, chat, delay=1.0):
    response = chat.send_message(prompt + context)
    time.sleep(delay)
    return response.text

# Sorgu üretme fonksiyonu
def generate_queries(rewrite_chat, query, delay=1.0):
    rewrite_prompt = f"""Bu metni analiz ederek, rüyada görülen ana unsurları belirle ve her bir unsur için uygun sorgular üret. Her sorgu, "Rüyada [Nesne/Varlık] [Eylem]" formatında olmalı ve ayrı bir satırda yazılmalıdır. Ayrıca, tanımlayıcı özellikleri içeren varyantlarını da ekle: {query}"""
    rewritten_query = generate_llm_answer(rewrite_prompt, "", rewrite_chat, delay)
    queries = []
    for line in rewritten_query.split('\n'):
        line = line.strip()
        if line and not line.lower().startswith("varyantlar") and "rüyada" in line.lower():
            if line.startswith("-"):
                line = line[1:].strip()
            if line.lower().startswith("rüyada") and line.endswith(("mek", "mak")):
                queries.append(line)
    print(f"------- Rüya Sorguları -------\n")
    for i, q in enumerate(queries, 1):
        print(f"Sorgu {i}: {q}")
    return queries

# Model yanıtı üretme fonksiyonu
def generate_model_answer(interpretation_chat, chroma_collection, model_name, query, queries, n_results=5, delay=1.0):
    query_results = []
    for q in queries:
        results = retrieve_docs(chroma_collection, q, n_results)
        ruya, yorum, dist = filter_relevant_result(results['documents'], results['metadatas'], results['distances'])
        query_results.append((q, ruya, yorum))

    print(f"\n------- {model_name} - Bulunan Rüyalar ve Yorumlar -------\n")
    for q, ruya, yorum in query_results:
        print(f"Sorgu: {q}")
        print(f"Rüya: {ruya}")
        print(f"Yorum: {yorum}\n")

    return query_results

# En iyi modeli seçme fonksiyonu
def select_best_model(chat, query_interpretations):
    prompt = "For the following queries, please select the retrieved dream that best matches the query.\n\n"
    model_names = ["DistilUSE", "BERT-Turkish", "PubMedBERT", "GIST"]
    for i, qi in enumerate(query_interpretations):
        prompt += f"Query {i+1}: {qi['query']}\n"
        for j, model_name in enumerate(model_names):
            ruya, _ = qi["models"].get(model_name, ("No interpretation found", ""))
            prompt += f"Model {j+1}: {ruya}\n"
        prompt += "\n"
    prompt += "Please respond with the model number (1, 2, 3, or 4) that has the best matching dream for each query, in the format:\n"
    prompt += "Query 1: [model number]\nQuery 2: [model number]\n..."

    response = chat.send_message(prompt)
    time.sleep(1.0)  # API rate limit için

    # Yanıtı parse et
    lines = response.text.strip().split('\n')
    selections = {}
    for line in lines:
        if line.startswith("Query "):
            parts = line.split(':')
            if len(parts) == 2:
                query_num = parts[0].strip().split(' ')[1]
                model_num = parts[1].strip()
                try:
                    selections[int(query_num)] = int(model_num)
                except ValueError:
                    pass  # Hatalı satırları atla
    return selections

# Kullanıcı dostu çıktı üreten fonksiyon
def generate_user_friendly_output(query, best_responses, interpretation_chat, delay=1.0):
    output = "Rüyanızda gördüklerinizi şöyle yorumlayabiliriz:\n\n"

    # Yorumları grupla
    yorum_gruplari = {}
    for q, (_, _, _, yorum) in best_responses.items():
        ruya_unsuru = q.replace('Rüyada', '').strip()  # "Rüyada" kelimesini çıkar
        if yorum not in yorum_gruplari:
            yorum_gruplari[yorum] = []
        yorum_gruplari[yorum].append(ruya_unsuru)

    # Her grup için bir satır oluştur
    for yorum, ruya_list in yorum_gruplari.items():
        ruya_str = ", ".join(ruya_list)
        output += f"**{ruya_str}**: {yorum}\n\n"

    # Genel yorum ekle
    context = "".join([f"{yorum}\n" for yorum in yorum_gruplari.keys()])
    general_prompt = f"Kullanıcının rüyası: '{query}'. Aşağıdaki tabirlere tam sadık kalarak, rüyanın genel bir yorumunu Türkçe olarak yap:\n{context}"
    general_comment = generate_llm_answer(general_prompt, "", interpretation_chat, delay)

    output += f"\n**Rüyanızın Genel Yorumu**:\n{general_comment}"
    return output

# Model karşılaştırma ve birleştirme fonksiyonu
def compare_models(query, interpretation_chat, rewrite_chat, delay=1.0):
    print("\n=== Rüya Sorguları Üretiliyor ===")
    queries = generate_queries(rewrite_chat, query, delay)

    # Model isimleri ve koleksiyonlar
    model_names = ["DistilUSE", "BERT-Turkish", "PubMedBERT", "GIST"]
    chroma_collections = [chroma_collection_distiluse, chroma_collection_bert_turkish,
                          chroma_collection_pubmed, chroma_collection_gist]

    # Her model için yorumları topla
    all_interpretations = {}
    for model_name, chroma_collection in zip(model_names, chroma_collections):
        interpretations = generate_model_answer(interpretation_chat, chroma_collection,
                                                model_name, query, queries, delay=delay)
        all_interpretations[model_name] = interpretations

    # Query_interpretations yapısını oluştur
    query_interpretations = []
    for i, q in enumerate(queries):
        query_dict = {"query": q, "models": {}}
        for model_name in model_names:
            if i < len(all_interpretations[model_name]):
                _, ruya, yorum = all_interpretations[model_name][i]
                query_dict["models"][model_name] = (ruya, yorum)
            else:
                query_dict["models"][model_name] = ("No interpretation found", "Bu sorgu için yorum yok.")
        query_interpretations.append(query_dict)

    # En iyi modelleri seç
    selections = select_best_model(interpretation_chat, query_interpretations)

    # Best_responses'u doldur
    best_responses = {}
    for i, q in enumerate(queries):
        selected_model_num = selections.get(i+1, 1)  # Varsayılan olarak Model 1
        model_name = model_names[selected_model_num - 1]
        ruya, yorum = query_interpretations[i]["models"][model_name]
        best_responses[q] = (model_name, q, ruya, yorum)

    # En iyi yanıtları konsola yazdır
    print("\n=== Seçilen En İyi Yanıtlar ===")
    for q, (model_name, _, ruya, yorum) in best_responses.items():
        print(f"Sorgu: {q}")
        print(f"Seçilen Model: {model_name}")
        print(f"Rüya: {ruya}")
        print(f"Yorum: {yorum}\n")

    # Kullanıcı dostu çıktı üret
    user_friendly_output = generate_user_friendly_output(query, best_responses, interpretation_chat, delay)
    print("\n=== Rüyanızın Yorumu ===")
    display(to_markdown(user_friendly_output))
    return user_friendly_output

# Ana program
collection_name_distiluse = "RuyaTabirleri_distiluse"
collection_name_bert_turkish = "RuyaTabirleri_bert_turkish"
collection_name_pubmed = "RuyaTabirleri_pubmed"
collection_name_gist = "RuyaTabirleri_gist"

file_path = '/content/son_guncellenmis_dosya.xlsx'

try:
    chroma_client_distiluse, chroma_collection_distiluse = load_existing_chromadb(
        collection_name_distiluse, "distiluse-base-multilingual-cased-v1", chromaDB_path
    )
except (ValueError, FileNotFoundError):
    print("Mevcut veritabanı bulunamadı veya boş. Yeni veri yüklenecek.")
    chroma_client_distiluse, chroma_collection_distiluse = load_excel_to_chromadb(
        collection_name_distiluse, "distiluse-base-multilingual-cased-v1", chromaDB_path, file_path
    )

try:
    chroma_client_bert_turkish, chroma_collection_bert_turkish = load_existing_chromadb(
        collection_name_bert_turkish, "emrecan/bert-base-turkish-cased-mean-nli-stsb-tr", chromaDB_path
    )
except (ValueError, FileNotFoundError):
    print("Mevcut veritabanı bulunamadı veya boş. Yeni veri yüklenecek.")
    chroma_client_bert_turkish, chroma_collection_bert_turkish = load_excel_to_chromadb(
        collection_name_bert_turkish, "emrecan/bert-base-turkish-cased-mean-nli-stsb-tr", chromaDB_path, file_path
    )

try:
    chroma_client_pubmed, chroma_collection_pubmed = load_existing_chromadb(
        collection_name_pubmed, "NeuML/pubmedbert-base-embeddings", chromaDB_path
    )
except (ValueError, FileNotFoundError):
    print("Mevcut veritabanı bulunamadı veya boş. Yeni veri yüklenecek.")
    chroma_client_pubmed, chroma_collection_pubmed = load_excel_to_chromadb(
        collection_name_pubmed, "NeuML/pubmedbert-base-embeddings", chromaDB_path, file_path
    )

try:
    chroma_client_gist, chroma_collection_gist = load_existing_chromadb(
        collection_name_gist, "avsolatorio/GIST-small-Embedding-v0", chromaDB_path
    )
except (ValueError, FileNotFoundError):
    print("Mevcut veritabanı bulunamadı veya boş. Yeni veri yüklenecek.")
    chroma_client_gist, chroma_collection_gist = load_excel_to_chromadb(
        collection_name_gist, "avsolatorio/GIST-small-Embedding-v0", chromaDB_path, file_path
    )

# Colab Secrets'tan API anahtarlarını al
API_KEYS = [
    userdata.get('GOOGLE_API_KEY_1'),
    userdata.get('GOOGLE_API_KEY_2'),
    userdata.get('GOOGLE_API_KEY_3'),
    userdata.get('GOOGLE_API_KEY_4')
]

# API anahtarlarının varlığını kontrol et
for i, key in enumerate(API_KEYS, 1):
    if not key:
        raise ValueError(f"GOOGLE_API_KEY_{i} Colab Secrets'ta bulunamadı. Lütfen ekleyin.")

# Sistem talimatları
system_prompt = """Sen bir islami rüya tabiri uzmanısın.
Sana verilen rüya ve yorum bağlamına dayanarak yalnızca bu bilgilerle yanıt ver.
Eğer bir unsur için yorum yoksa, bunu açıkça belirt.
Yanıtlarını Türkçe olarak, doğal, akıcı ve anlaşılır bir şekilde ver."""

rewrite_system_prompt = """Sen bir yardımcı asistansın. Görevin, kullanıcının verdiği rüya metnini analiz ederek, rüyada görülen ana unsurları (nesneler, varlıklar, eylemler) belirlemek ve her bir unsur için uygun sorgular üretmektir. Her sorgu, "Rüyada [Nesne/Varlık] [Eylem]" formatında olmalı ve eylemler genel olarak '-mek' veya '-mak' ekleriyle bitmelidir (örneğin, Görmek, Atmak, Konuşmak). Her sorgu ayrı bir satırda yazılmalıdır.

Ayrıca, her unsur için tanımlayıcı özellikleri (renk, boyut, durum vb.) içeren tüm varyantlarını da üret. Cümlenin içerisinden ayrı unsurları da kelime kelime varyantlarıyla tek tek çıkar. Örneğin:
- "Büyük yeşil yılan" için:
  - Rüyada Yılan Görmek
  - Rüyada Yeşil Yılan Görmek
  - Rüyada Büyük Yılan Görmek
- "Kağıt gibi bir şeye sarılıyordum" için:
  - Rüyada Sarılmak
  - Rüyada Kağıda Sarılmak
  - Rüyada Kağıt Görmek
- "Kar üstünde giden yılan" için:
  - Rüyada Yılan Görmek
  - Rüyada Kar Görmek
  - Rüyada Yılanın Kar Üstünde Gitmesi

Yalnızca rüyada görülen unsurları dahil et; gerçek hayatla ilgili detayları (örneğin 'gerçekte', 'bu aralar', 'ne manaya geliyor') çıkar. Metnin anlamını değiştirmeden ve yorum eklemeden, yalnızca rüya ile ilgili ana unsurları, tüm varyantlarını, ayrıca eş anlamlarını ve benzer anlamlarının hepsini bu formatta yaz. Burası çok önemli tüm varyantlarını yazmalısın."""

# Kullanıcılar için chatbotları oluştur
num_users = 4
user_chats = []

for i in range(num_users):
    api_key = API_KEYS[i % len(API_KEYS)]  # Round-robin ile API anahtarı atama
    interpretation_chat = build_chatbot(system_prompt, api_key)
    rewrite_chat = build_chatbot(rewrite_system_prompt, api_key)
    user_chats.append({
        "interpretation_chat": interpretation_chat,
        "rewrite_chat": rewrite_chat
    })

# Ana döngü
print("Rüya Tabiri Chatbot'una Hoş Geldiniz!")
print("Rüyanızı doğal bir şekilde anlatın, size en güzel yorumu sunalım.")
user_id = 0
while True:
    print(f"\nKullanıcı {user_id + 1}, lütfen rüyanızı girin (çıkmak için 'çık'): ")
    soru = input()
    if soru.lower() == "çık":
        print("Görüşmek üzere!")
        break
    interpretation_chat = user_chats[user_id]["interpretation_chat"]
    rewrite_chat = user_chats[user_id]["rewrite_chat"]
    compare_models(soru, interpretation_chat, rewrite_chat, delay=1.0)
    user_id = (user_id + 1) % num_users  # Bir sonraki kullanıcıya geç

chromadb yüklendi.
sentence_transformers yüklendi.
google-generativeai yüklendi.
pandas zaten yüklü.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Mevcut koleksiyon yüklendi: 35586 öğe.
Mevcut koleksiyon yüklendi: 35586 öğe.
Mevcut koleksiyon yüklendi: 35586 öğe.
Mevcut koleksiyon yüklendi: 35586 öğe.
Rüya Tabiri Chatbot'una Hoş Geldiniz!
Rüyanızı doğal bir şekilde anlatın, size en güzel yorumu sunalım.

Kullanıcı 1, lütfen rüyanızı girin (çıkmak için 'çık'): 

=== Rüya Sorguları Üretiliyor ===
------- Rüya Sorguları -------

Sorgu 1: Rüyada Otobüs Beklemek
Sorgu 2: Rüyada Otobüs Görmek
Sorgu 3: Rüyada Kimse Görmemek
Sorgu 4: Rüyada Gece Görmek
Sorgu 5: Rüyada Saate Bakmak
Sorgu 6: Rüyada Saat Görmek
Sorgu 7: Rüyada Işık Görmek
Sorgu 8: Rüyada Bisikletli Çocuk Görmek
Sorgu 9: Rüyada Bisiklet Görmek
Sorgu 10: Rüyada Çocuk Görmek
Sorgu 11: Rüyada Uzaktan Işık Görmek
Sorgu 12: Rüyada Yaklaşan Işık Görmek
Sorgu 13: Rüyada Beklemek
Sorgu 14: Rüyada Bisiklet Sürmek
Sorgu 15: Rüyada Vakit Görmek
Sorgu 16: Rüyada Zaman Görmek
Sorgu 17: Rüyada Karanlıkta Beklemek
Sorgu 18: Rüyada Işık Hüzmesi Görmek
Sorgu 19: Rüyada Ümit Etmek
Sorgu 20: R

> Rüyanızda gördüklerinizi şöyle yorumlayabiliriz:
> 
> **Otobüs Beklemek**: Rüyada otobüs ya da benzeri her hangi bir araç beklemek, rüyayı gören kişinin yaşayacağı bir musibet karşısında, yapacağı hiçbir şey olmadığı için elinin kolunun bağlı olacağına, bu durumun da kendini kahredeceğine delalet eder. Kişi, yine de çareler aramaktan vazgeçmeyecek demektir.
> 
> **Otobüs Görmek**: Rüyada otobüs görmek rüya sahibi için yol anlamına gelir. Aynı zamanda uzun zamandan beri görüşmediği biriyle bir araya gelmesine ya da tesadüfen rastlaşmasına delalet eder. Eğer rüya sahibi kendini otobüsün içinde bir yere giderken görürse bu rüya sahibinin bir haksızlığa uğrayacağına ve boş yere suçlanacağına işaret eder. Eğer rüya sahibi sıkışık bir otobüste ise bu onun kalabalık bir ortama gireceği anlamına gelir. Rüyada otobüs görmek münakaşa anlamına da gelmektedir. Gereksiz yere çıkacak olan bir ağız münakaşasına ya da kavgaya delalet eder. Rüyada otobüs görülmesi genel olarak hem ayrılığa hem de bir araya gelmeye yorumlanır. Rüya sahibi kendini bir otobüsün içinde daha önce hiç gitmediği bir yerde görürse bu hayırlı işlere işaret eder. Rızık, kısmet, kazanç ve yeni iş anlamına gelir. Rüya sahibi otobüsü seyir halinde görürse ahlaksız bir kimseye ya da hoş olmayan davranışlara işaret eder.
> 
> **Kimse Görmemek**: Rüyada kimlik görmek , rüyayı gören kişinin hayatını baştan sona kadar değiştirmek istediğine ve bunun için yollar aradığına tabir edilir. Rüya sahibinin yaşam tarzını değiştireceğine, yeni işlere atılacağına, başka başka yerlere seyahat edeceğine, yeni dostluklar ve arkadaşlıklar edineceğine, geçmişini arkada bırakacağına ve yaşamında yeni bir sayfa açacağına rivayet edilir.
> 
> **Gece Görmek**: Rüyada gece görmek, kişinin iki fikirlilik edeceğine yani daha açık bir tabirle kararsızlık yaşayacağına, bu sürede hem vakit hem de nakit kaybederek zarar edeceğine yorulur.
> 
> 
> 
> 
> **Saate Bakmak**: Rüyada saate bakmak , kişinin içindeki enerjiyi, hırsı ve arzuları ortaya çıkarabilmek için şartların olgunlaşmasını beklediğini ancak bu süreçte erken harekete geçmesi yüzünden beklemediği sonuçlar alabileceğini işaret eder. Saate bakmak heyecan dolu bir bekleyiş içine girileceğine, özellikle evlilik için gün sayılacağına da alamettir. Zamana bağlı işleri olan rüya sahibinin içindeki heyecan ve sabırsızlığa da işaret eden rüya, keyifli, merak içinde geçen bir sürecin de tabiridir. Saate bakmak istikrarlı bir büyüme içinde olan iş yeri sahiplerinin, geleceklerini çok iyi programlayacaklarına ve her işlerinin belirledikleri üzere harfiyen devam edeceğine, sorunsuz bir çalışma hayatına ve her an daha da artan bir gelire sahip olmaya da delalettir.
> 
> **Saat Görmek**: Rüyada saat görmek kişinin sözüne bağlı olmasına, hayatın her anına değer vermesine, buluşma saatine sadık olmasına bazen de güvenilmeyecek, yalancı, söylediklerini yerine getirmeyen kişiye delalet eder. Rüyada saat görmek can sıkıcı, moral bozucu ve kişiyi üzüntüye sürükleyecek sözler işitmek anlamına gelir. Rüyada saati ayarladığını gören kişinin belli bir düzeni olacak ve her şey yolunda gidecek demektir. Rüyada saat sesi duymak bir davet ya da teklif anlamına gelir. Rüyada saatin çalması kötü sözler duymaya işaret eder. Rüyada çok şık ve güzel bir saat görmek kişinin hayatında meydana gelecek olan hoş gelişmelerin ve sürprizlerin habercisidir. Rüyada duvardaki ya da bir yerde sabit duran bir saatin çalışması hane içindeki saadetin simgesidir.
> 
> **Işık Görmek, Uzaktan Işık Görmek, Yaklaşan Işık Görmek, Işık Hüzmesi Görmek**: Rüyada ışık görmek , rüya sahibi için aydınlık, ferah, mutlu ve güzel günlerin yaklaştığına delalet eder. Rüya sahibi gelecek günlerde kurtuluşa erecek ve çektiği tüm zahmetten ve çileden kurtulacaktır. Rüyada ışık görmek, her türlü acının dinmesi ve her türlü yaranın iyileşip kapanması anlamına gelir. Rüya sahibi, geçmişte yaşadığı tüm kötü tecrübelerden, acılardan ve sıkıntılardan, onları kafasından atmak suretiyle tamamen uzaklaşacak, yaşadığı günlerin tadını çıkarmaya başlayacak demektir. Rüyasında ışık göre kişiye Allah sabır, kolaylık ve sakinlik verecektir. Rüya sahibi doğru yoldan ayrılmadığı sürece Allah kendisinden yana olacak ve işlerin en çıkmaza girdiği anda bile kendisine el uzatarak onu yaşadığı cehennemden ve karanlıktan çekip kurtaracaktır.
> 
> **Bisikletli Çocuk Görmek**: Rüyayı gören kişinin gönül hoşluğu, hafiflik ve huzur içinde olacağına, hayatına heyecan ve renk gelecek gelişmeler yaşayacağına, çevre değişikliği yaparak eğlenebileceği ortamlara gireceğine ve kafa dengi arkadaşlar edineceğine rivayet edilir.
> 
> **Bisiklet Görmek**: Rüyada bisiklet görmek zorluğun, sıkıntının, üzüntünün ve kaosun verdiği karamsar ruh halinden arınmaya, tazelenmeye ve hayat bulmaya işaret eder. Rüyada bisiklete bindiğini gören kişi önemli ve ciddi bir sorunun üstesinden gelecek ve büyük bir başarı elde ederek, hem gururlanacağı hem de huzur bulacağı bir iş yapacak demektir. Rüyada bisiklet görmek aynı zamanda kişinin geleceği ile ilgili bir konuda hayati bir karar vermesi gerekeceğine işaret eder. Bir kişi eğer rüyasında bisiklete binip, dik bir rampayı çıkmaya çalıştığını görüyorsa bu kalan hayatının aydınlık, mutlu ve başarılı olacağına işaret eder. Rüyada bisiklet görmek kişinin yapacağı yanlış şeylerden ders çıkaracağı anlamına gelirken, aynı zamanda kişinin yanlış kararlar vermekten ve hatalar yapmaktan kendini alamadığına işaret eder.
> 
> **Çocuk Görmek**: Rüyada çocuk görmek , nasıl görüldüğüne ve gören kişiye bağlı olarak bir çok şekilde yorumlanabilir. Ancak genelde çocuk görmek gören kişinin sevineceği şeklinde yorumlanır. Birden çok çocuk görmek, özellikle de güzel çocuklarsa bir yerden para geleceğini gösterir. Bir kadının rüyasında çocuk görmesi, o kadının anne olma isteğine işaret ediyor olabilir. Çocuk aynı zamanda haber almaya işarettir.
> Erkek çocuk görüldüyse iyi haber, kız çocuk görüldüyse biraz sıkıntılı ve üzücü bir haber alınabilir. Rüyada gerçek hayatta tanımadığınız yabancı bir çocuk görmeniz bir düşmanınız olduğunu gösterir. Tanımadığınız çocuğu rüyanızda nasıl gördüğünüze bağlı olarak, düşmanınızın şu anki durumu ile ilgili yorum yapabilmek mümkündür. Örneğin, rüyanızda tanımadığınız hasta bir çocuk görüyorsanız, düşmanınız kötü bir durumda olabilir.
> 
> **Beklemek**: Rüyada beklemek , düşüncelerin olgunlaşmasına, işlerin vaktine ermesine, zamanı gelenlerin bir bir yaşanmaya ve gerçek olmaya başlamasına tabir edilir. Bu rüya gerçek anlamına yakın şekilde tabir edilir ve rüya sahibinin hayatla ilgili beklentilerinin, hayallerinin ve isteklerinin gerçekleşmesi için sabır göstermesi anlamına gelir. Bazen de rüya sahibinin kendini olmadık, hayırsız ve kötü işlerin içinde bulabileceği şeklinde yorumlanır. Rüyada beklemek, ticaret ehli kimseler için rızık ve kazanç bolluğu ile tabir edilir.
> 
> **Bisiklet Sürmek**: Edilen duaların Allah katında karşılığını bulması sayesinde uzun zamandan beri içinde bulunulan sıkıntıların ve sorunların yakın bir zamanda ortadan kaldırılacağına, maddi olarak çok büyük bir zorlanmaya sebebiyet veren işin nihayete erdirileceğine, sevilen bir kişinin yolundan gidileceğine ve o kişi gibi büyük başarılara imza atılacağına delalettir. Rüyada yokuşta bisiklet sürmek, zor bir işin üstesinden gelineceği, yapılan hataların telafi edileceği, verilen sözlerin zaman kaybetmeden yerine getirileceği, yakın bir arkadaşın sıkıntısını gidermesine yardımcı olunacağı ve hem aile hayatında hem de iş hayatında çok güzel günlerin yaklaştığı anlamına gelir.
> 
> **Vakit Görmek**: Rüyada vakit görmek , beklenen bir haber olduğuna ya da kişinin hayatı ile ilgili beklentilerinin yüksek olduğuna işarettir. Sabırsız davranıldığı için yaşanan sıkıntıların daha da artacağına, iş konularında aşırı rahat davranıldığı için sıkışmaya başlandığına da alamettir. Zamanında atılmayan adımların veya çözülmeyen sorunların rüya sahibinin moralini bozacağına ve hayata olan bakış açısını değiştirmek zorunda kalacağına, aldığı sorumlulukları yerine getirmek için her zamankinden daha fazla çalışacağına da yorumlanır. Eğer rüyada görülen vakit hatırlanıyorsa o şekilde yorumlanması en doğrusu olacaktır.
> 
> **Zaman Görmek**: Rüyada zaman görmek de tarih görmeye benzer biçimde yorumlanır. Gelecekten haber vereceği gibi geçmişte olmuş bir olaya da dikkat çekmek olarak açıklanır.
> 
> **Karanlıkta Beklemek**: Rüyada sıra beklemek , kurulan hayallerin gerçek olması için şartların henüz olgun hale gelmediğini ve daha fazla çaba gösterilmesi gerektiğinin işaretidir. İş anlamında kişilerin durgunluk sürecine gireceklerini ve maddi olarak fazla açılmadan, büyük işlere girmeden belli bir süre sabır göstermeleri gerektiğini de bildirir. Hak edilen mutluluğun, rızkın ve işsizler için istenilen işin elde edilmesine vakit olduğunu, beklentilerin gerçekleşmesi için zamana ihtiyaç olduğunu da tabir eder. Sıra beklemek hamile kalındığının haberini almak şeklinde de yorumlanır ve genellikle kişinin hayatının her alanında bir durgunluk dönemi yaşayacağını, fakat bu durumun kısa süreceğini işaret eder.
> 
> **Ümit Etmek**: Rüyada ümit etmek , kötü günlerin yavaşça geride kalacağını ve kişinin sabrının sınandığı bu dönemde biraz daha tevekkül içinde olması gerektiğini ifade eder. Yaşananlara gönül gözüyle bakacak olan rüya sahibinin hayatındaki her detayda kendisi için iyi bir yan, bir hayır bulacağına, mutluluğu uzakta aramaktan vazgeçeceğine delalet eder.  Olumuz bir durum üzerine umut edildiğini görmek kişinin iradesini zorlayacak günler geçireceğini ve özellikle iş yaşantısında karşılaşacağı bazı güçlükler karşısında zorlanacağını, bu dönemde inançlarını sorgulamaya başlayacağını, önceliklerini değiştireceğini işaret eder. Bir izdivaç veya beraberlikle alakalı ümit edildiğini görmek ise iyi gelişmelere, kişinin eskisine göre daha anlayışlı ve yumuşak başlı biri olacağına yorumlanır.
> 
> **Umutsuz Beklemek**: Rüyada umutsuzluk görmek , uzun zamandan beri sorunlu bir şekilde devam eden çalışmalar yüzünden çok sıkıntılı ve zor durumlara düşüleceğine, atılan adımların maddi ve manevi olarak çok zarar vereceğine, ortaklık kurulan kişilerle büyük bir tartışma yaşanacağına, gelinen son noktaya kadar çok büyük sıkıntılar çekilmesine rağmen işlerin türlü zora gireceğine ve sağlık sorunları yüzünden epey zorluk yaşanacağına alamet eder.
> 
> **Tek Başına Beklemek**: Rüyada tek başına kalmak, borçların artmasına, çekilen üzüntü nedeni ile bazı hastalıkların baş göstermesine, huzursuzluğun rüya sahibini yiyip bitirmesine, kişinin yakasını bir türlü beladan kurtaramamasına delalet eder.
> 
> **Issız Bir Yerde Beklemek**: Çok büyük bir sıkıntıda olduğunuza ve neredeyse bunalıma girmek üzere olduğunuza alamettir. Rüyada zifiri karanlık, iş, aile ve duygusal yaşantınızda hemen hemen hiçbir şeyin yolunda gitmediğine siz sıkıntıların üstesinden gelmeye çalıştıkça başarısız olduğunuza işaret etmektedir. Aynı zamanda hayatınız ile ilgili yapacağınız yenilikler için iyi bir zaman olmadığı ve girişeceğiniz işlerde istediğiniz başarıyı yakalayamayacağınız anlamına da gelmektedir.
> 
> **Karanlıkta Işık Görmek**: Rüyasında gecenin ortasında ışık gören kişi içinde bulunduğu sıkıntılı durumdan çabucak kurtulur. Hastaysa şifa bulur, suçluysa aklanır, kısırsa evlat sahibi olur. Bu rüyada herkese yarar bir hayır mutlaka bulunur çünkü kerametli rüyalar arasındadır ve herkese nasip olmayacağı şeklinde tabir edilir.
> 
> 
> 
> 
> 
> **Rüyanızın Genel Yorumu**:
> Rüyada otobüs beklemek, yaşayacağınız bir musibet karşısında elinizin kolunuzun bağlı olacağına ve çaresizlik hissedeceğinize işaret ediyor. Otobüsün gelmemesi, bu durumun uzun süreceğini ve sizi kahredeceğini gösteriyor. Etrafta kimsenin olmaması, bu zorluğu tek başınıza aşmak zorunda kalacağınız anlamına geliyor. Gece yarısı olması, bu sıkıntının karanlık ve belirsiz bir dönemde ortaya çıkacağına işaret ediyor. Uzaktan görünen ışık, bir umut ışığı gibi beliriyor ancak bisikletli bir çocuk olarak gelip geçmesi, beklediğiniz çözümün veya yardımın tam olarak istediğiniz gibi olmayacağını, geçici bir ferahlama sağlayacağını fakat kalıcı bir çözüm getirmeyeceğini gösteriyor. Rüya, sabırlı olmanız, tevekkül etmeniz ve küçük de olsa gelen yardımlara değer vermeniz gerektiğini vurguluyor.



Kullanıcı 2, lütfen rüyanızı girin (çıkmak için 'çık'): 

=== Rüya Sorguları Üretiliyor ===
------- Rüya Sorguları -------

Sorgu 1: Rüyada Otobüs Beklemek
Sorgu 2: Rüyada Otobüs Görmek
Sorgu 3: Rüyada Gelmeyen Otobüs Görmek
Sorgu 4: Rüyada Saat Görmek
Sorgu 5: Rüyada Saate Bakmak
Sorgu 6: Rüyada Gece Yarısı Görmek
Sorgu 7: Rüyada Işık Görmek
Sorgu 8: Rüyada Uzaktan Işık Görmek
Sorgu 9: Rüyada Yaklaşan Işık Görmek
Sorgu 10: Rüyada Bisiklet Görmek
Sorgu 11: Rüyada Bisikletli Çocuk Görmek
Sorgu 12: Rüyada Geçip Gitmek

------- DistilUSE - Bulunan Rüyalar ve Yorumlar -------

Sorgu: Rüyada Otobüs Beklemek
Rüya: Rüyada Otobüs Beklemek
Yorum: Rüyada otobüs ya da benzeri her hangi bir araç beklemek, rüyayı gören kişinin yaşayacağı bir musibet karşısında, yapacağı hiçbir şey olmadığı için elinin kolunun bağlı olacağına, bu durumun da kendini kahredeceğine delalet eder. Kişi, yine de çareler aramaktan vazgeçmeyecek demektir.

Sorgu: Rüyada Otobüs Görmek
Rüya: Rüyada Otobüs Görmek
Yorum: Rüyad

...


Kullanıcı 3, lütfen rüyanızı girin (çıkmak için 'çık'): 

=== Rüya Sorguları Üretiliyor ===
------- Rüya Sorguları -------

Sorgu 1: Rüyada Otobüs Beklemek
Sorgu 2: Rüyada Otobüs Görmek
Sorgu 3: Rüyada Saat Görmek
Sorgu 4: Rüyada Saate Bakmak
Sorgu 5: Rüyada Gece Yarısı Görmek
Sorgu 6: Rüyada Işık Görmek
Sorgu 7: Rüyada Uzaktan Işık Görmek
Sorgu 8: Rüyada Bisiklet Görmek
Sorgu 9: Rüyada Bisikletli Çocuk Görmek
Sorgu 10: Rüyada Beklemek
Sorgu 11: Rüyada Gelmemek
Sorgu 12: Rüyada Bakmak
Sorgu 13: Rüyada Görünmek
Sorgu 14: Rüyada Yaklaşmak
Sorgu 15: Rüyada Geçmek
Sorgu 16: Rüyada Gitmek
Sorgu 17: Rüyada Çocuk Görmek
Sorgu 18: Rüyada Bisiklet Sürmek
Sorgu 19: Rüyada Gece Görmek
Sorgu 20: Rüyada Yarısı Görmek
Sorgu 21: Rüyada Bisikletli Görmek
Sorgu 22: Rüyada Uzak Görmek
Sorgu 23: Rüyada Işık Saçmak

------- DistilUSE - Bulunan Rüyalar ve Yorumlar -------

Sorgu: Rüyada Otobüs Beklemek
Rüya: Rüyada Otobüs Beklemek
Yorum: Rüyada otobüs ya da benzeri her hangi bir araç beklemek, rüyayı göre

> Rüyanızda gördüklerinizi şöyle yorumlayabiliriz:
> 
> **Otobüs Beklemek**: Rüyada otobüs ya da benzeri her hangi bir araç beklemek, rüyayı gören kişinin yaşayacağı bir musibet karşısında, yapacağı hiçbir şey olmadığı için elinin kolunun bağlı olacağına, bu durumun da kendini kahredeceğine delalet eder. Kişi, yine de çareler aramaktan vazgeçmeyecek demektir.
> 
> **Otobüs Görmek**: Rüyada otobüs görmek rüya sahibi için yol anlamına gelir. Aynı zamanda uzun zamandan beri görüşmediği biriyle bir araya gelmesine ya da tesadüfen rastlaşmasına delalet eder. Eğer rüya sahibi kendini otobüsün içinde bir yere giderken görürse bu rüya sahibinin bir haksızlığa uğrayacağına ve boş yere suçlanacağına işaret eder. Eğer rüya sahibi sıkışık bir otobüste ise bu onun kalabalık bir ortama gireceği anlamına gelir. Rüyada otobüs görmek münakaşa anlamına da gelmektedir. Gereksiz yere çıkacak olan bir ağız münakaşasına ya da kavgaya delalet eder. Rüyada otobüs görülmesi genel olarak hem ayrılığa hem de bir araya gelmeye yorumlanır. Rüya sahibi kendini bir otobüsün içinde daha önce hiç gitmediği bir yerde görürse bu hayırlı işlere işaret eder. Rızık, kısmet, kazanç ve yeni iş anlamına gelir. Rüya sahibi otobüsü seyir halinde görürse ahlaksız bir kimseye ya da hoş olmayan davranışlara işaret eder.
> 
> **Saat Görmek**: Rüyada saat görmek kişinin sözüne bağlı olmasına, hayatın her anına değer vermesine, buluşma saatine sadık olmasına bazen de güvenilmeyecek, yalancı, söylediklerini yerine getirmeyen kişiye delalet eder. Rüyada saat görmek can sıkıcı, moral bozucu ve kişiyi üzüntüye sürükleyecek sözler işitmek anlamına gelir. Rüyada saati ayarladığını gören kişinin belli bir düzeni olacak ve her şey yolunda gidecek demektir. Rüyada saat sesi duymak bir davet ya da teklif anlamına gelir. Rüyada saatin çalması kötü sözler duymaya işaret eder. Rüyada çok şık ve güzel bir saat görmek kişinin hayatında meydana gelecek olan hoş gelişmelerin ve sürprizlerin habercisidir. Rüyada duvardaki ya da bir yerde sabit duran bir saatin çalışması hane içindeki saadetin simgesidir.
> 
> **Saate Bakmak**: Rüyada saate bakmak , kişinin içindeki enerjiyi, hırsı ve arzuları ortaya çıkarabilmek için şartların olgunlaşmasını beklediğini ancak bu süreçte erken harekete geçmesi yüzünden beklemediği sonuçlar alabileceğini işaret eder. Saate bakmak heyecan dolu bir bekleyiş içine girileceğine, özellikle evlilik için gün sayılacağına da alamettir. Zamana bağlı işleri olan rüya sahibinin içindeki heyecan ve sabırsızlığa da işaret eden rüya, keyifli, merak içinde geçen bir sürecin de tabiridir. Saate bakmak istikrarlı bir büyüme içinde olan iş yeri sahiplerinin, geleceklerini çok iyi programlayacaklarına ve her işlerinin belirledikleri üzere harfiyen devam edeceğine, sorunsuz bir çalışma hayatına ve her an daha da artan bir gelire sahip olmaya da delalettir.
> 
> **Gece Yarısı Görmek, Gece Görmek**: Rüyada gece görmek, kişinin iki fikirlilik edeceğine yani daha açık bir tabirle kararsızlık yaşayacağına, bu sürede hem vakit hem de nakit kaybederek zarar edeceğine yorulur.
> 
> 
> 
> 
> **Işık Görmek, Uzaktan Işık Görmek, Işık Saçmak**: Rüyada ışık görmek , rüya sahibi için aydınlık, ferah, mutlu ve güzel günlerin yaklaştığına delalet eder. Rüya sahibi gelecek günlerde kurtuluşa erecek ve çektiği tüm zahmetten ve çileden kurtulacaktır. Rüyada ışık görmek, her türlü acının dinmesi ve her türlü yaranın iyileşip kapanması anlamına gelir. Rüya sahibi, geçmişte yaşadığı tüm kötü tecrübelerden, acılardan ve sıkıntılardan, onları kafasından atmak suretiyle tamamen uzaklaşacak, yaşadığı günlerin tadını çıkarmaya başlayacak demektir. Rüyasında ışık göre kişiye Allah sabır, kolaylık ve sakinlik verecektir. Rüya sahibi doğru yoldan ayrılmadığı sürece Allah kendisinden yana olacak ve işlerin en çıkmaza girdiği anda bile kendisine el uzatarak onu yaşadığı cehennemden ve karanlıktan çekip kurtaracaktır.
> 
> **Bisiklet Görmek, Bisikletli Görmek**: Rüyada bisiklet görmek zorluğun, sıkıntının, üzüntünün ve kaosun verdiği karamsar ruh halinden arınmaya, tazelenmeye ve hayat bulmaya işaret eder. Rüyada bisiklete bindiğini gören kişi önemli ve ciddi bir sorunun üstesinden gelecek ve büyük bir başarı elde ederek, hem gururlanacağı hem de huzur bulacağı bir iş yapacak demektir. Rüyada bisiklet görmek aynı zamanda kişinin geleceği ile ilgili bir konuda hayati bir karar vermesi gerekeceğine işaret eder. Bir kişi eğer rüyasında bisiklete binip, dik bir rampayı çıkmaya çalıştığını görüyorsa bu kalan hayatının aydınlık, mutlu ve başarılı olacağına işaret eder. Rüyada bisiklet görmek kişinin yapacağı yanlış şeylerden ders çıkaracağı anlamına gelirken, aynı zamanda kişinin yanlış kararlar vermekten ve hatalar yapmaktan kendini alamadığına işaret eder.
> 
> **Bisikletli Çocuk Görmek**: Rüyayı gören kişinin gönül hoşluğu, hafiflik ve huzur içinde olacağına, hayatına heyecan ve renk gelecek gelişmeler yaşayacağına, çevre değişikliği yaparak eğlenebileceği ortamlara gireceğine ve kafa dengi arkadaşlar edineceğine rivayet edilir.
> 
> **Beklemek**: Rüyada beklemek , düşüncelerin olgunlaşmasına, işlerin vaktine ermesine, zamanı gelenlerin bir bir yaşanmaya ve gerçek olmaya başlamasına tabir edilir. Bu rüya gerçek anlamına yakın şekilde tabir edilir ve rüya sahibinin hayatla ilgili beklentilerinin, hayallerinin ve isteklerinin gerçekleşmesi için sabır göstermesi anlamına gelir. Bazen de rüya sahibinin kendini olmadık, hayırsız ve kötü işlerin içinde bulabileceği şeklinde yorumlanır. Rüyada beklemek, ticaret ehli kimseler için rızık ve kazanç bolluğu ile tabir edilir.
> 
> **Gelmemek**: Rüyada görememek , aşırı hayalci bir tutum içinde olmak ve kendini tabiri caizse fildişi bir kuleye hapsetmek nedeniyle etrafında olan biteni doğru dürüst kavrayamamak, olaylara gereken zamanda müdahale edememek anlamına gelir. Kişinin içinde bulunduğu bu durum özellikle iş konularında mali kayıpların yaşanabileceğinin altını çizer ve rehavete kapılmanın sonuçlarının kişiyi hüsrana uğratabileceğini bildirir. Uyarıcı rüyalar arasında olduğu için, bu rüyayı gören kimselerin daha uyanık olmaları ve işlerini ciddiye almaları gerekir. Aynı zamanda dost ve düşman ayrımının iyi yapılması gerektiğini, bilhassa bugünlerde çok yakın olunmayan veya yeterince tanınmayan kişilere sır vermemek gerektiğini de hatırlatır. Bencil ve çıkarcı kimselerden oluşan bir ortama veya gruba girmek, bu kişilere uyum sağlayarak sorumluluklarını ihmal etmek manasına da gelir.
> 
> **Bakmak**: Rüyada bakmak konusu oldukça detaylı bir konudur. Rüyanın yorumu, rüyanın görülme şekline bağlı olarak farklı anlamlara gelebilir. Örneğin; bir kişi rüyasında her hangi bir eşyaya çok beğenerek ya da imrenerek bakarsa çok istediği bir mala kavuşur. Eğer kişi rüyada iki yana doğru baktığını görürse o halde kaybetme kaygısı taşıdığı ve çok sevdiği bazı kişiler olduğu anlamına gelir.
> 
> **Görünmek**: Rüyada görümce görmek rüya sahibinin başta anne-baba olmak üzere tüm hane halkından ve yakın çevresinden hakarete uğrayacağı ve kınanacağı bir şey yapacağına delalet eder. Rüyada görümce görmek çok iyi şekilde yorumlanmaz, tatsızlık, kavga, dedikodu, kötü haber ve kargaşa anlamına gelir.
> Bazı rüya tabirlerine göre eş tarafından bir kadını ve yine eş tarafından bir erkeği rüyada birlikte görmek evlat sahibi olmaya delalet eder. Görümce, insanın çevresindeki kuşkucu, kuruntulu, huysuz, mutsuz ve kavgacı kimselere işaret ettiği gibi rüya sahibinin de her şeyden kuşku duyan ve öküzün altında buzağı arayan, bir bardak suda fırtınalar koparan kişi olduğu yorumu yapılır. Bazı rüya tabirlerine göre görümce kişinin toplum için güzel ve hayırlı şeyler yaparak dilden dile dolaşan bir üne sahip olacağı ve hayır duaları göreceği şeklinde yorumlanır.
> 
> **Yaklaşmak**: Rüyada yakınlaşmak , sevilen kişilerden biriyle ilgili çok kötü ve üzücü bir haber alınacağına, bu haber yüzünden uzun zamandan beri üzerinde çalışılan çalışmaların ve projelerin ertelenmesine ya da rafa kaldırılmasına sebep olacağına, maddi açıdan büyük bir zarara uğranacağına, büyük umutlarla girilen bir işin sorunlarla devam edeceğine ve endişeli bir bekleyişin başlayacağına işaret eder.
> 
> **Geçmek**: Rüyada geçmek , zorlu sınavları aşmak ve bugüne kadar verilen emeklerin neticesini almaya başlamak demektir. Nereden veya ne şekilde geçildiğini görmek rüyanın anlamını belirlediğinden dolayı önemlidir. Rüyada ateş içinden geçmek kötü günler atlatılacağına veya ölümden dönülecek kazalar ve hastalıklarla mücadele edileceğine alamettir. Bir çemberin içinden geçtiğini gören kimselerin ailelerinin içinde karışıklıklar meydana gelir ve eski hesaplar yeniden açılır. Laf, söz nedeniyle tüm ilişkilerin zarar göreceğine yorumlanır. Denizden yürüyerek geçmek mucizevi bir durumla karşılaşmak, bebeği olmayanların hamile kalması olarak tabir edilirken, nehirden karşıya yüzerek geçtiğini görmek, büyük bir başarının habercisidir. Yoldan karşıya geçmek ise rüya sahibinin kalbinin doğruluğuna, niyetinin saflığına ve çevresince çok sevilen biri olduğuna alamet eder.
> 
> **Gitmek**: Rüyada gitmek , rüyayı gören kişinin, iş hayatında ve aile hayatında çok uzun zamandan beri hayal ettiği şeylerin yakın bir zaman içinde herhangi bir sorunla karşılaşmadan gerçekleşeceğine, sıkıntılarının ve sorunlarının sona ereceğine, aile hayatı içinde yaşadığı sorunların kısa bir zamanda biteceğine ve iş hayatında yapılacak çok büyük ve hayırlı yatırımlar sayesinde çok büyük kazançlar elde edeceğine delalet eder.
> 
> **Çocuk Görmek**: Rüyada çocuk görmek , nasıl görüldüğüne ve gören kişiye bağlı olarak bir çok şekilde yorumlanabilir. Ancak genelde çocuk görmek gören kişinin sevineceği şeklinde yorumlanır. Birden çok çocuk görmek, özellikle de güzel çocuklarsa bir yerden para geleceğini gösterir. Bir kadının rüyasında çocuk görmesi, o kadının anne olma isteğine işaret ediyor olabilir. Çocuk aynı zamanda haber almaya işarettir.
> Erkek çocuk görüldüyse iyi haber, kız çocuk görüldüyse biraz sıkıntılı ve üzücü bir haber alınabilir. Rüyada gerçek hayatta tanımadığınız yabancı bir çocuk görmeniz bir düşmanınız olduğunu gösterir. Tanımadığınız çocuğu rüyanızda nasıl gördüğünüze bağlı olarak, düşmanınızın şu anki durumu ile ilgili yorum yapabilmek mümkündür. Örneğin, rüyanızda tanımadığınız hasta bir çocuk görüyorsanız, düşmanınız kötü bir durumda olabilir.
> 
> **Bisiklet Sürmek**: Edilen duaların Allah katında karşılığını bulması sayesinde uzun zamandan beri içinde bulunulan sıkıntıların ve sorunların yakın bir zamanda ortadan kaldırılacağına, maddi olarak çok büyük bir zorlanmaya sebebiyet veren işin nihayete erdirileceğine, sevilen bir kişinin yolundan gidileceğine ve o kişi gibi büyük başarılara imza atılacağına delalettir. Rüyada yokuşta bisiklet sürmek, zor bir işin üstesinden gelineceği, yapılan hataların telafi edileceği, verilen sözlerin zaman kaybetmeden yerine getirileceği, yakın bir arkadaşın sıkıntısını gidermesine yardımcı olunacağı ve hem aile hayatında hem de iş hayatında çok güzel günlerin yaklaştığı anlamına gelir.
> 
> **Yarısı Görmek**: Rüyada yarış görmek , iki farklı şekilde yorumlanır. Eğer kişi girdiği yarışı kazanmışsa işlerinin her geçen gün daha iyi bir hal alacağına, sıkıntılarının ve sorunlarının sona ereceğine, mutlu günler göreceğine, hayırlı ve sevinçli haberler alacağına, girdiği büyük işlerde çok iyi noktalara geleceğine, iş hayatının ve aile hayatının bolluk ve bereket içinde geçeceğine işaret eder. Eğer girilen yarış kaybedilmişse sorunların artacağına, hayırlı bir şekilde ilerleyen işlerin sekteye uğrayacağına, sağlık sorunları yaşanacağına ve huzur kaçıracak olaylar yaşanacağına rivayet eder
> 
> **Uzak Görmek**: Rüyada uzak görmek , kısa bir süre için de olsa sevdiklerinden ayrı kalmak zorunda olan rüya sahibinin özlem duygusuyla baş etmekte zorlanacağının ifadesidir. Kendisine yakın olan kimselere olan düşkünlüğe ve aileye karşı aşırı hassas tavırlara yorumlanır. Rüya sahibinin anne ve babasına olan ilgisinin bu dönemde daha da yoğunlaşacağına, onları memnun etmek için var gücüyle çalışacağını ve birlikte daha çok vakit geçirmeye özen göstereceğini işaret eder. Evlilikler için eşlerin aralarında esecek soğuk rüzgarların bir süre daha devam edeceğine alamettir.
> 
> 
> **Rüyanızın Genel Yorumu**:
> Rüyanız, hayatınızda beklediğiniz bir şeylerin gerçekleşmemesi ve bu durumun sizde yarattığı hayal kırıklığı ile ilgili. Otobüs beklemek, umutla beklediğiniz bir olayın veya fırsatın bir türlü gelmemesi, eliniz kolunuzun bağlı hissetmenize ve çaresizlik içinde olmanıza işaret ediyor. Gece yarısı olması, bu bekleyişin uzun sürdüğünü ve umutsuzluğa kapılmaya başladığınızı gösteriyor.
> 
> Uzaktan görünen ışık, bir umut ışığı gibi beliriyor ancak bisikletli bir çocuğun geçip gitmesi, beklediğiniz çözümün veya fırsatın aslında size ait olmadığını veya beklentilerinizi karşılamayacağını gösteriyor. Bu durum, hayata dair beklentilerinizin gerçekleşmesi için sabırlı olmanız gerektiğine işaret ederken, bazen de yanlış beklentiler içinde olabileceğinizi ve farklı yönlere bakmanız gerektiğini hatırlatıyor.
> 
> Rüyanız, hayatınızda bir dönüm noktasında olduğunuzu ve geleceğinizle ilgili önemli kararlar vermeniz gerektiğini de gösteriyor. Beklentilerinizi gözden geçirerek, sabırlı olmaya ve doğru adımlar atmaya odaklanmanız gerekiyor.



Kullanıcı 4, lütfen rüyanızı girin (çıkmak için 'çık'): 

=== Rüya Sorguları Üretiliyor ===
------- Rüya Sorguları -------

Sorgu 1: Rüyada Ev Görmek
Sorgu 2: Rüyada Kapı Görmek
Sorgu 3: Rüyada Evin Kapısını Görmek
Sorgu 4: Rüyada Kapıyı Açmaya Çalışmak
Sorgu 5: Rüyada Anahtar Görmek
Sorgu 6: Rüyada Ses Duymak
Sorgu 7: Rüyada Bağırmak
Sorgu 8: Rüyada Cevap Alamamak
Sorgu 9: Rüyada Uyanmak
Sorgu 10: Rüyada Kapıyı Açmak
Sorgu 11: Rüyada Evin Kapısını Açmak
Sorgu 12: Rüyada Anahtar ile Kapı Açmak
Sorgu 13: Rüyada Anahtar ile Evin Kapısını Açmak
Sorgu 14: Rüyada Anahtar Aramak
Sorgu 15: Rüyada Kilit Görmek
Sorgu 16: Rüyada Kilit Açmak
Sorgu 17: Rüyada Kilit Kırmak
Sorgu 18: Rüyada Evin Kapısı Görmek
Sorgu 19: Rüyada Sesler Duymak
Sorgu 20: Rüyada Birisi Görmek
Sorgu 21: Rüyada Bağırışmak
Sorgu 22: Rüyada Cevap Vermek
Sorgu 23: Rüyada Çözülmek
Sorgu 24: Rüyada Uyanmak
Sorgu 25: Rüyada Evin Kapısını Açmaya Çalışmak
Sorgu 26: Rüyada "Kim o?" Diye Bağırmak

------- DistilUSE - Bulunan Rüyalar

> Rüyanızda gördüklerinizi şöyle yorumlayabiliriz:
> 
> **Ev Görmek**: Bu rüya esasen çok detaylıdır ve rüyanın tam yorumu evin şekline ve durumuna bağlı olarak yapılmalıdır fakat genel bir yorumla söyle açıklanmaktadır; rüyada ev görmek, dünya hayatından sonra yaşanacak olan hayata alamet eder. Günahların hesabının sorulacağı ve sevapların mükâfatlarının alınacağı yaşama işaret eder.
> 
> 
> 
> 
> **Kapı Görmek**: Rüyada kapı görmek, çeşitli şekillerde tabir edilir. Kapının nerede olduğuna ya da nerenin kapısı olduğuna bağlı olarak yorumlanır ama genel anlamı ile bu rüya, hayatı yoluna koyan, cefakâr bir kimse ile açıklanır. Evin annesi ya da hanımı olarak değerlendirilir.
> 
> **Evin Kapısını Görmek, Evin Kapısı Görmek**: Rüyada görülen ev kapısı, kişinin dayanacağı ve sığınacağı önemli bir adamdır. Ev reisine, malını kazandığı kişiye, ekmek kapısına da yorulur. Rüyada ev kapısının kırılması, kişinin başına gelecek belalar ve sıkıntılardır. Rüyasında evinin kapısının kaybolduğunu gören kişi, ibadetlerden yüz çevirir, iyilik kapıları kendisine kapanır.
> 
> **Kapıyı Açmaya Çalışmak, Kapıyı Açmak, Evin Kapısını Açmak, Evin Kapısını Açmaya Çalışmak**: Kapalı bir kapıyı açmak gönül almak, ailevi sorumluluklar edinmek, genç yaşta zengin olmak ve iyi bir gelecek için doğru yolda yürümek anlamındadır. Hayırlı bir rüyadır ve evlenmek, çocuk sahibi olmak gibi anlamlara da gelir. Yeni bir evin kapısını açtığını gören kişiler gerçek hayatta da güzel bir ev satın alırlar. Eski bir evin kapısını açmak ise ana baba ziyaretinde bulunmak ya da ölmüşler için hayır dağıtmak şeklinde yorumlanır.
> 
> **Anahtar Görmek**: Rüyada anahtar görmek tek bir şekilde yorumlanmaz. Rüya sahibi elindeki anahtarla eğer bir yeri açabiliyorsa bu iyiye kapatıyorsa bu da kötüye işaret eder. Rüyada görülen anahtar bir dileğin ya da rüya sahibinin olmasını çok istediği bir şey için Allah’a yakarışın işareti olarak kabul edilir. Rüyada salt bir anahtar görmek dini vecibeleri yerine getirmeye işaret eder. Anahtarın elden düşürülmesi, dini vecibeleri tamamen bırakıldığı anlamına gelmez ama rüya sahibinin dini görevlerini yaparken isteyerek yapılmadığı anlamına gelir. Anahtar tek başına görüldüğünde çok hayırlı şeylere işaret eder. Sağlığa, huzura, müjdeli haberlere, hayırlı çocuklara, kazanca, helal rızka gibi, bir kimse rüyasında anahtar aldığını görürse bu onun büyük bir varlığa ve zenginliğe kavuşacağı anlamına gelir. Rüyada birden fazla anahtar görülmesi tek bir anahtar görülmesinden kat be kat daha iyidir. Birden fazla anahtar gören kişinin büyük bir servete sahip olacağı anlamına gelir.
> 
> **Ses Duymak, Sesler Duymak**: Rüyada ses duymak , rüya sahibinin arkasından çok iyi anılacağına, öleceği zaman kendisini yaşatacak güzellikte işler yapacağına, herkesin gönül rızasını alarak adım atacağına delalet eder. Rüya sahibi, insanlar için iyi ve faydalı şeyler yapmak adına çırpınan, buna büyük oranda enerji ve para harcayan kişi olarak değerlendirilir. Rüya sahibinin amacı insanlık adına faydalı olacak bir iş yapmak, ölünce arkasından güzel ve hayırlı bir şeyler bırakmaktır.
> 
> **Bağırmak, Bağırışmak**: Rüyada bağırmak rüya sahibinin hayatı boyunca şahit olacağı ve yanı başında yaşanacak olaylar olarak tabir edilir. Kişi toplumda her zaman sesi çıkan ve meramını anlatmaktan çekinmeyen bir kimse olarak daima sivrilen kişi olacaktır. Rüyada bağırdığını ve aynı zamanda eli ve kolu ile jestler yaptığını gören kişi toplum yararına olacak bir hareketin öncülüğünü yapacak diye ifade edilir.
> Rüyada ulu orta bağırdığını gören kişinin devlet kapısında, kâğıt üzerinde bir işi olacak diye yorumlanır. Rüyada kişinin gereksiz yere ya da kendi kendine boşluğa bağırması, o kişinin deveyi pire yapan bu nedenle de herkesi gereksiz yere galeyana ya da telaşa düşüren kimse olarak tabir edilir.
> 
> **Cevap Alamamak**: Kişi için içinde bulunduğu yolun doğru olmadığına ve aldığı kararların onu hiçbir yere götürmeyeceğine, yanlış seçimler yüzünden maddi kayıpları kadar manevi anlamda da kayıplar yaşayacağına, mevcut düzeninin bozulmasından dolayı da psikolojik olarak kötü bir duruma düşeceğine alamet eder.
> 
> **Uyanmak**: Rüyada uyanmak , rüya sahibinin bulunduğu şehri hatta ülkeyi değiştirmesine, yıllardır çalıştığı işi bırakmasına ve evini satmasına yani çılgınca şeyler yapmasına yorumlanır. Rüyayı gören kişinin, yapmaktan sıkıldığı şeyleri bırakmasına, yeni şeyler denemesine ve hayatını neredeyse tamamen değiştirmesine delalet eder. Arkadaş çevresinde uzun süredir ahbaplık edilen kişilerle aradaki bağları koparmak anlamına da gelir.
> 
> **Anahtar ile Kapı Açmak, Anahtar ile Evin Kapısını Açmak**: Rüyasında anahtarla herhangi bir kapı açan kimse yeni bir işe başlar. Bu rüyayı gören kişi çok hayırlı olacak, maddi anlamda da oldukça iyi getirisi olan, güzel bir işe girer. Anahtarla kapı açmak daima güzel ve hayırlı bir rüyadır; kapatmak ise tam tersidir. Kapı kapatıp kilitleyen kişinin hayatında önemli değişiklikler olur. Bu, işini kaybetme, önüne çıkan fırsatları tepme anlamlarına da gelebilmektedir.
> 
> **Anahtar Aramak**: Rüyada anahtar aramak , çıkış yolu bulmak için çabalayan rüya sahibinin henüz ihtiyacı olan cevapları alamayacağını ve kendisine yardım eli uzanması için sabırla beklemesi gerektiğine işaret eder. Hayallerine ulaşmak için uğraşan kişinin kendisini bir anda umutsuzluğa düşmüş olarak bulabileceği anlamına da gelir. İş yerindeki sorunlara delalet olan rüya kişinin hayatını daha sakin ve rahat bir şekilde sürdürmek için başka yollar bulmaya çalışacağını da bildirir. Anahtar aramak yıllardır kişinin içinde yaşattığı bir şüpheye veya kuşku yüzünden çevresine karşı her daim sorgulayan gözlerle baktığını da işaret eder.
> 
> **Kilit Görmek**: Rüyada kilit görmek, itibarlı kişi olmaya, kariyerini ve koltuğunu korumaya, hakkını yedirmemeye aynı zamanda hak yememeye, malını ve kazancını garanti altına almaya, özü sözü bir mert ve güvenilir kişi olmaya tabir edilir.
> 
> **Kilit Açmak**: Rüyada kilit açmak, rüyayı gören kişinin işinde ileri gideceğine, yaşadığı zararı ve mali krizi atlatacağına, rakiplerine karşı güç ve kudret sahibi olup, onların önüne geçeceğine delalet eder. Makam sahibi olmaya, kötü günleri atlatmaya ve aydınlık günleri görmeye yorulur.
> 
> 
> 
> 
> **Kilit Kırmak**: Rüyasında kilitlenmiş bir yeri kilidi kırarak açmak, kişinin zorla yapacağı bir işe, meşakkat çekerek ulaşacağı bir kazanca işarettir. Rüyada herhangi bir kilidi kıran kişi, kendisini çekemeyen ve kötülüğünü isteyen birtakım adamların şerrinden emin olur. Bu rüya dişinizi tırnağınıza takarak çalışıp çabalayarak bir lütfa ve nimete erişeceğinize yorumlanır.
> 
> **Birisi Görmek**: Rüyada birini görmek , eğer tanıdık biri görülürse rüyayıs gören kişinin, bu kişiye çok büyük maddi ve manevi destekte bulunacağına, sorunlarının çözümü için canla başla çalışacağına, iş hayatında çok büyük bir yardımda bulunacağına ve aile hayatı ile ilgili bir sorunu çözeceğine delalet eder. Rüya sahibi eğer tanımadığı birini görmüşse, kötü ve sıkıntılı durumların yaklaşmakta olduğuna, çok büyük maddi ve manevi zararlara uğranacağına ve çok kötü insanlar ile muhatap olmak zorunda kalınacağına tabir edilir.
> 
> **Cevap Vermek**: Rüyada soru cevaplamak da, soru çözmek gibi ele alınır ve hayırlara alamet ettiği şeklinde kabul edilir. Rüyayı gören kişinin başarısız olduğu konularda bilgi ve beceri edineceğine bu sayede işlerinde ve yaşamında büyük oranda iyileşme kaydedeceğine, yol alacağına ve zaferler elde edeceğine tabir edilir.
> 
> **Çözülmek**: Kurtuluşun başlangıcı olarak kabul edilir. Kolaylığa ve imkânlara kavuşmaya, güzel fırsatlar yakalamaya, büyük kısmetler almaya, sorunları, zorlukları ve sıkıntıları aşmak için güç elde etmeye, dertlerin dermanını bulmaya ve çarelere kavuşmaya alamet eder.
> 
> **"Kim o?" Diye Bağırmak**: Rüyada korkmak ve bağırmak, mutluluk ile nara atmaya işaret eder. Rüya sahibi zafer ve büyük başarı kazandığı bir işin haberini alacak, sevinçten çılgına dönecektir. Bu iş kişiye çok uğurlu gelecek ve kendisine başka kısmet kapıları da açacaktır.
> 
> 
> 
> 
> 
> 
> **Rüyanızın Genel Yorumu**:
> Rüyanız, hayatınızdaki belirsizlikler, çözülmemiş sorunlar ve bunlarla başa çıkma çabanızla ilgili önemli mesajlar içeriyor. İşte rüyanızın genel bir yorumu:
> 
> **Ev ve Kapı:** Rüyada ev, genellikle kişinin iç dünyasını, güvende hissettiği alanı veya yaşamının önemli bir bölümünü temsil eder. Kapı ise, yeni başlangıçlara, fırsatlara veya önemli kararlara açılan bir geçit olarak yorumlanabilir. Evinizin kapısını açmaya çalışmanız, hayatınızda yeni bir aşamaya geçmek veya bir sorunu çözmek için çaba gösterdiğinizi gösteriyor.
> 
> **Anahtarın Dönmemesi:** Anahtarın dönmemesi, şu anda bazı engellerle karşılaştığınızı, çözüm yollarının tıkalı olduğunu veya bir konuda ilerleme kaydetmekte zorlandığınızı işaret ediyor olabilir. Bu, iş hayatınızda, ilişkilerinizde veya kişisel gelişiminizde karşınıza çıkan bir engel olabilir.
> 
> **İçeriden Gelen Sesler ve "Kim O?" Diye Bağırmak:** İçeriden gelen sesler, bilinçaltınızdaki endişeleri, şüpheleri veya çözülmemiş sorunları temsil edebilir. "Kim o?" diye bağırmanız ve cevap alamamanız, bu sorunlarla yüzleşmek istediğinizi ancak net bir cevap veya çözüm bulamadığınızı gösteriyor. Bu, içsel bir arayışınızın veya belirsizliklerle dolu bir durumun işareti olabilir.
> 
> **Kilidin Birden Çözülmesi:** Kilidin birden çözülmesi, umutsuzluğa kapıldığınız bir anda beklenmedik bir şekilde bir çözümün ortaya çıkabileceğini veya bir engelin ortadan kalkabileceğini gösteriyor. Bu, sabırlı olmanız ve çabalarınızın karşılığını alacağınız anlamına gelebilir.
> 
> **Uyanmak:** Rüyadan uyanmak, gerçek hayatta da bir uyanış yaşayabileceğinizi, yeni bir farkındalık kazanabileceğinizi veya bir sorunun üstesinden gelebilecek gücü kendinizde bulabileceğinizi işaret ediyor.
> 
> **Genel Yorum:** Rüyada yaşadığınız bu deneyim, hayatınızda çözülmeyi bekleyen bazı sorunlar olduğunu ve bu sorunlarla yüzleşmek için çaba göstermeniz gerektiğini gösteriyor. Ancak, umutsuzluğa kapılmamanız ve sabırlı olmanız önemlidir. Çünkü beklenmedik bir anda bir çözüm ortaya çıkabilir ve hayatınızda yeni bir sayfa açabilirsiniz. Rüya, aynı zamanda içsel arayışınızın ve belirsizliklerle dolu bir durumun işareti olabilir, bu nedenle kendinizi dinlemeniz ve iç sesinizi takip etmeniz önemlidir.



Kullanıcı 1, lütfen rüyanızı girin (çıkmak için 'çık'): 
